In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
import os
os.chdir("./drive/My Drive/intel_intern/dataset")

In [ ]:
import pickle
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import chi2
import numpy as np

In [ ]:
path_df = "News_train_data.pickle"

with open(path_df, 'rb') as data:
    df = pickle.load(data)

df.head()

,Id,Text,Category,id,News_length
0,1,Telecom service provider Bharti Airtel has tak...,ai,1,1439
1,2,Founded in 2018 by Jitesh Dadlani and Sandip S...,ai,1,1043
2,3,"SirionLabs, which provides artificial intellig...",ai,1,1400
3,4,In a bid to raise awareness on its products na...,ai,1,1323
4,5,"Eventila Technologies Pvt. Ltd, which operates...",ai,1,1637


In [ ]:
df.loc[1]['Text']

"Founded in 2018 by Jitesh Dadlani and Sandip Singh, Ishitva uses deep-tech such as artificial intelligence (AI), machine learning (ML), and IoT to effectively segregate waste to recover recyclables, thus creating circular economy. In an interview with YourStory, the founders said that the startup's proprietary device’s automatic sorting system helps improve both quantity, as well as the quality of recycling activities. This in turn, reduces dumping and burning of dry waste such as scraps, paper, plastics films, bags, containers, and plastic bottles etc.Ishitva’s machine uses AI, robotics, and ML to decipher the composition of waste, and sort them into various categories under recyclables and non-recyclable labels.Ishitva currently offers a range of solutions including SUKA (AI-powered air sorting), YUTA, (AI-powered robotic sorting), Netra AI Vision system, which helps identify waste, and smart bins. The AI-powered Netra identifies the waste, learns about its contents by capturing ima

In [ ]:
df['Content_Parsed_1'] = df['Text'].str.replace("\r", " ")
df['Content_Parsed_1'] = df['Content_Parsed_1'].str.replace("\n", " ")
df['Content_Parsed_1'] = df['Content_Parsed_1'].str.replace("    ", " ")

handling quoted text

In [ ]:
df['Content_Parsed_1'] = df['Content_Parsed_1'].str.replace('"', '')
df['Content_Parsed_1'] = df['Content_Parsed_1'].str.replace('”', '')
df['Content_Parsed_1'] = df['Content_Parsed_1'].str.replace('“', '')


changing to lowecase

In [ ]:
df['Content_Parsed_2'] = df['Content_Parsed_1'].str.lower()

In [ ]:
punctuation_signs = list("?:!.,;")
df['Content_Parsed_3'] = df['Content_Parsed_2']

for punct_sign in punctuation_signs:
    if(punct_sign=='.'):
      df['Content_Parsed_3'] = df['Content_Parsed_3'].str.replace(punct_sign, " ")
    df['Content_Parsed_3'] = df['Content_Parsed_3'].str.replace(punct_sign, '')

handling possesive pronouns

In [ ]:
df['Content_Parsed_4'] = df['Content_Parsed_3'].str.replace("'s", "")
df['Content_Parsed_4'] = df['Content_Parsed_3'].str.replace("’s", "")
df['Content_Parsed_4'] = df['Content_Parsed_3'].str.replace("'", "")
df['Content_Parsed_4'] = df['Content_Parsed_3'].str.replace("’", "")

In [ ]:
nltk.download('punkt')
print("------------------------------------------------------------")
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
------------------------------------------------------------
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [ ]:
wordnet_lemmatizer = WordNetLemmatizer()
nrows = len(df)
lemmatized_text_list = []

for row in range(0, nrows):
    
    # Create an empty list containing lemmatized words
    lemmatized_list = []
    
    # Save the text and its words into an object
    text = df.loc[row]['Content_Parsed_4']
    text_words = text.split(" ")

    # Iterate through every word to lemmatize
    for word in text_words:
        lemmatized_list.append(wordnet_lemmatizer.lemmatize(word, pos="v"))
        
    # Join the list
    lemmatized_text = " ".join(lemmatized_list)
    
    # Append to the list containing the texts
    lemmatized_text_list.append(lemmatized_text)

In [ ]:
df['Content_Parsed_5'] = lemmatized_text_list

stop words

In [ ]:
!python -m nltk.downloader stopwords

/usr/lib/python3.6/runpy.py:125: RuntimeWarning: 'nltk.downloader' found in sys.modules after import of package 'nltk', but prior to execution of 'nltk.downloader'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
# Loading the stop words in english
stop_words = list(stopwords.words('english'))
stop_words[0:10]

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're"]

In [ ]:
example = "me eating a meal"
word = "me"

# The regular expression is:
regex = r"\b" + word + r"\b"  # we need to build it like that to work properly

re.sub(regex, "StopWord", example)

'StopWord eating a meal'

In [ ]:
df['Content_Parsed_6'] = df['Content_Parsed_5']

for stop_word in stop_words:

    regex_stopword = r"\b" + stop_word + r"\b"
    df['Content_Parsed_6'] = df['Content_Parsed_6'].str.replace(regex_stopword, '')

In [ ]:
df.loc[1]['Text']

"Founded in 2018 by Jitesh Dadlani and Sandip Singh, Ishitva uses deep-tech such as artificial intelligence (AI), machine learning (ML), and IoT to effectively segregate waste to recover recyclables, thus creating circular economy. In an interview with YourStory, the founders said that the startup's proprietary device’s automatic sorting system helps improve both quantity, as well as the quality of recycling activities. This in turn, reduces dumping and burning of dry waste such as scraps, paper, plastics films, bags, containers, and plastic bottles etc.Ishitva’s machine uses AI, robotics, and ML to decipher the composition of waste, and sort them into various categories under recyclables and non-recyclable labels.Ishitva currently offers a range of solutions including SUKA (AI-powered air sorting), YUTA, (AI-powered robotic sorting), Netra AI Vision system, which helps identify waste, and smart bins. The AI-powered Netra identifies the waste, learns about its contents by capturing ima

In [ ]:
df.loc[1]['Content_Parsed_6']

"found  2018  jitesh dadlani  sandip singh ishitva use deep-tech   artificial intelligence (ai) machine learn (ml)  iot  effectively segregate waste  recover recyclables thus create circular economy    interview  yourstory  founder say   startup' proprietary devices automatic sort system help improve  quantity  well   quality  recycle activities    turn reduce dump  burn  dry waste   scrap paper plastics film bag containers  plastic bottle etc ishitvas machine use ai robotics  ml  decipher  composition  waste  sort   various categories  recyclables  non-recyclable label ishitva currently offer  range  solutions include suka (ai-powered air sorting) yuta (ai-powered robotic sorting) netra ai vision system  help identify waste  smart bin   ai-powered netra identify  waste learn   content  capture image   scan  recyclable material "

In [ ]:
list_columns = ["Id", "Category", "Text", "Content_Parsed_6"]
df = df[list_columns]

df = df.rename(columns={'Content_Parsed_6': 'Content_Parsed'})
df.head()

,Id,Category,Text,Content_Parsed
0,1,ai,Telecom service provider Bharti Airtel has tak...,telecom service provider bharti airtel take ...
1,2,ai,Founded in 2018 by Jitesh Dadlani and Sandip S...,found 2018 jitesh dadlani sandip singh ishi...
2,3,ai,"SirionLabs, which provides artificial intellig...",sirionlabs provide artificial intelligence te...
3,4,ai,In a bid to raise awareness on its products na...,bid raise awareness products namely fasta...
4,5,ai,"Eventila Technologies Pvt. Ltd, which operates...",eventila technologies pvt ltd operate artifi...


In [ ]:
category_codes = {
    'ai': 0,
    'iot': 1,
    'industry': 2,
    'cloud': 3,
    'dc': 4,
    'telecom': 5,
    'cc': 6
}

In [ ]:
# Category mapping
df['Category_Code'] = df['Category']
df = df.replace({'Category_Code':category_codes})

In [ ]:
df.head()

,Id,Category,Text,Content_Parsed,Category_Code
0,1,ai,Telecom service provider Bharti Airtel has tak...,telecom service provider bharti airtel take ...,0
1,2,ai,Founded in 2018 by Jitesh Dadlani and Sandip S...,found 2018 jitesh dadlani sandip singh ishi...,0
2,3,ai,"SirionLabs, which provides artificial intellig...",sirionlabs provide artificial intelligence te...,0
3,4,ai,In a bid to raise awareness on its products na...,bid raise awareness products namely fasta...,0
4,5,ai,"Eventila Technologies Pvt. Ltd, which operates...",eventila technologies pvt ltd operate artifi...,0


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df['Content_Parsed'], 
                                                    df['Category_Code'], 
                                                    test_size=0.15, 
                                                    random_state=8)

In [ ]:
# Parameter election
ngram_range = (1,2)
min_df = 10
max_df = 1.
max_features = 300

In [ ]:
tfidf = TfidfVectorizer(encoding='utf-8',
                        ngram_range=ngram_range,
                        stop_words=None,
                        lowercase=False,
                        max_df=max_df,
                        min_df=min_df,
                        max_features=max_features,
                        norm='l2',
                        sublinear_tf=True)
                        
features_train = tfidf.fit_transform(X_train).toarray()
labels_train = y_train
print(features_train.shape)

features_test = tfidf.transform(X_test).toarray()
labels_test = y_test
print(features_test.shape)

(331, 300)
(59, 300)


In [ ]:
from sklearn.feature_selection import chi2
import numpy as np

for Product, category_id in sorted(category_codes.items()):
    features_chi2 = chi2(features_train, labels_train == category_id)
    indices = np.argsort(features_chi2[0])
    feature_names = np.array(tfidf.get_feature_names())[indices]
    unigrams = [v for v in feature_names if len(v.split(' ')) == 1]
    bigrams = [v for v in feature_names if len(v.split(' ')) == 2]
    print("# '{}' category:".format(Product))
    print("  . Most correlated unigrams:\n. {}".format('\n. '.join(unigrams[-5:])))
    print("  . Most correlated bigrams:\n. {}".format('\n. '.join(bigrams[-2:])))
    print("")

# 'ai' category:
  . Most correlated unigrams:
. health
. learn
. intelligence
. artificial
. ai
  . Most correlated bigrams:
. intelligence ai
. artificial intelligence

# 'cc' category:
  . Most correlated unigrams:
. lenovo
. smartphone
. game
. laptops
. pc
  . Most correlated bigrams:
. market share
. per cent

# 'cloud' category:
  . Most correlated unigrams:
. google
. compute
. enterprises
. amazon
. cloud
  . Most correlated bigrams:
. market share
. google cloud

# 'dc' category:
  . Most correlated unigrams:
. store
. policy
. center
. centre
. data
  . Most correlated bigrams:
. data center
. data centre

# 'industry' category:
  . Most correlated unigrams:
. smart
. cities
. operate
. advance
. vehicles
  . Most correlated bigrams:
. google cloud
. data centre

# 'iot' category:
  . Most correlated unigrams:
. device
. connect
. smart
. things
. iot
  . Most correlated bigrams:
. data centre
. internet things

# 'telecom' category:
  . Most correlated unigrams:
. reliance


In [ ]:
# X_train
with open('data_pickles/X_train.pickle', 'wb') as output:
    pickle.dump(X_train, output)
    
# X_test    
with open('data_pickles/X_test.pickle', 'wb') as output:
    pickle.dump(X_test, output)
    
# y_train
with open('data_pickles/y_train.pickle', 'wb') as output:
    pickle.dump(y_train, output)
    
# y_test
with open('data_pickles/y_test.pickle', 'wb') as output:
    pickle.dump(y_test, output)
    
# df
with open('data_pickles/df.pickle', 'wb') as output:
    pickle.dump(df, output)
    
# features_train
with open('data_pickles/features_train.pickle', 'wb') as output:
    pickle.dump(features_train, output)

# labels_train
with open('data_pickles/labels_train.pickle', 'wb') as output:
    pickle.dump(labels_train, output)

# features_test
with open('data_pickles/features_test.pickle', 'wb') as output:
    pickle.dump(features_test, output)

# labels_test
with open('data_pickles/labels_test.pickle', 'wb') as output:
    pickle.dump(labels_test, output)
    
# TF-IDF object
with open('data_pickles/tfidf.pickle', 'wb') as output:
    pickle.dump(tfidf, output)